In [15]:
# Load libraries
import time
import ee
import geemap
import os
import numpy
import pandas as pd
import ee.mapclient
import pyarrow as pa
from datetime import datetime

In [16]:
# Connect to GEE
ee.Authenticate()
ee.Initialize(project = 'clearing-blackwaters')

# Load in Sentinel datasets
sent = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

# Standardize band names between the various collections and aggregate them into one image collection
bn = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12', 'QA60']
bns = ['Blue', 'Green', 'Red', 'Nir', 'Swir1', 'Swir2', 'QA60']

sent2 = sent.select(bn, bns)

# Set cloud cover threshold
sentinel = sent2 \
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50)) \

# Load in flowline of interest
all_flowlines = ee.data.listAssets({"parent": "projects/clearing-blackwaters/assets"})

# Get flowline ids
flowlinesID = []
for x in range(0, len(all_flowlines['assets'])):
    flowline_id = all_flowlines['assets'][x]['id']
    flowlinesID.append(flowline_id)

# Function to buffer each feature
def buffer_feature(feature):
    return feature.buffer(100, 10)

# Cloud mask
def cloud_mask(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = (
        qa.bitwiseAnd(cloud_bit_mask)
        .eq(0)
        .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    )
  
    return image.updateMask(mask).divide(10000)

In [ ]:
# Loops through every 2 weeks until current time

errors = list()

for i in range(0, len(flowlinesID)): 

    # Select individual flowline
    flowline = ee.FeatureCollection(flowlinesID[i])

    # Get all comids 
    flowline_comids = flowline.aggregate_array('comid').getInfo()
    
    # Loop through comids 
    for x in range(0, len(flowline_comids)): 
        
        # get individual comid
        flowline_fc = flowline.filter(ee.Filter.eq('comid', flowline_comids[x]))
 
        # Map the buffer function over the collection
        this_line = flowline_fc.map(buffer_feature)

        count = 1
        date = ee.Date("2019-01-01")
        while (ee.Date(datetime.now()).millis()).gte(date.advance(2, "week").millis()).getInfo() == 1: 
            
            # Filter Sentinel image to comid 
            sentinel_masked = sentinel.filterDate(date, date.advance(13, "days")).filterBounds(this_line).map(cloud_mask)
                  
            # Pull data for image
            try: 
                data = sentinel_masked.getRegion(this_line, 10)
                keys = ee.List(data.get(0)).getInfo()
                data_info = data.getInfo() 

                # Change save directory as needed
                save_path = "D:/dwz6/Documents/sentinel2_data/" + str(flowline_comids[x]) + " (" + str(count) + ").feather"
                
                df = pd.DataFrame(data_info[1:len(data_info)], columns = keys)
                df['SWM'] = (df['Blue'] + df['Green']) / (df['Nir'] + df['Swir1'])
                df.to_feather(save_path)
            except: 
                # Print errors and add to list
                error_comid = "comid: " + str(flowline_comids[x]) + " (" + str(count) + ") (asset: " + str(flowlinesID[i]) + ")"
                errors.append(error_comid)

            date = date.advance(2, "week")
            count += 1